In [2]:
import pandas as pd

# Load the data
data_path = "D:/prepared_data.csv"
data = pd.read_csv(data_path)

# Inspect the data
print(data.head())


                                          input  \
0          What is the process to file a claim?   
1       How can I check the status of my claim?   
2  What documents are required to file a claim?   
3     How long does it take to process a claim?   
4                    Can I file a claim online?   

                                            response  
0  To file a claim, please visit our website and ...  
1  You can check the status of your claim by logg...  
2  The required documents to file a claim include...  
3  The processing time for a claim varies dependi...  
4  Yes, you can file a claim online by visiting o...  


In [3]:
print(data.columns)

Index(['input', 'response'], dtype='object')


In [4]:
import torch # type: ignore
print(torch.__version__)


2.3.1+cu118


In [5]:
'''from transformers import GPT2Tokenizer
from datasets import Dataset

# Initialize the tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Combine the input and response into a single text column
data['text'] = data['input'] + " " + data['response']

tokenizer.pad_token = tokenizer.eos_token

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

# Convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(data[['text']])
dataset = dataset.map(tokenize_function, batched=True)

# Split the data into training and validation sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']'''


'from transformers import GPT2Tokenizer\nfrom datasets import Dataset\n\n# Initialize the tokenizer\nmodel_name = "gpt2"\ntokenizer = GPT2Tokenizer.from_pretrained(model_name)\n\n# Combine the input and response into a single text column\ndata[\'text\'] = data[\'input\'] + " " + data[\'response\']\n\ntokenizer.pad_token = tokenizer.eos_token\n\n# Tokenize the data\ndef tokenize_function(examples):\n    return tokenizer(examples[\'text\'], padding="max_length", truncation=True, max_length=512)\n\n# Convert to Hugging Face Dataset format\ndataset = Dataset.from_pandas(data[[\'text\']])\ndataset = dataset.map(tokenize_function, batched=True)\n\n# Split the data into training and validation sets\ntrain_test_split = dataset.train_test_split(test_size=0.1)\ntrain_dataset = train_test_split[\'train\']\nval_dataset = train_test_split[\'test\']'

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

# Load the CSV data
data = pd.read_csv('D:/prepared_data.csv')

# Initialize the GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

# Combine the input and response into a single text column
data['text'] = data['input'] + " " + data['response']


# Tokenize the data
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Convert data to Dataset format
dataset = Dataset.from_pandas(data[['text']])
dataset = dataset.map(tokenize_function, batched=True)

# Split the dataset into train and validation sets
train_dataset = dataset.shuffle(seed=42).select(range(int(0.8 * len(dataset))))
val_dataset = dataset.shuffle(seed=42).select(range(int(0.8 * len(dataset)), len(dataset)))

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()


Map:   0%|          | 0/126 [00:00<?, ? examples/s]

  0%|          | 0/75 [00:00<?, ?it/s]

d:\Anaconda\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:650: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Perplexity: {eval_results['eval_loss']}")


  0%|          | 0/7 [00:00<?, ?it/s]

Perplexity: 0.6973289847373962


In [ ]:
# Define the response generation function
def generate_response(input_text):
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    attention_mask = (inputs != tokenizer.pad_token_id).long()
    outputs = model.generate(inputs, attention_mask=attention_mask, max_length=512, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the response generation
input_text = "How can I reset my password?"
response = generate_response(input_text)
print(response)

NameError: name 'tokenizer' is not defined